# Psi4 RT-TDDFT Coupling Tutorial

"
            "This notebook demonstrates how to couple Meep to the Psi4-based RT-TDDFT driver"
            " via MaxwellLink's socket interface using the HCN example system.

**Prerequisites**

"
            "- `psi4` installed with Python bindings
"
            "- `pymeep` / `meep` (MPI-enabled build recommended)
"
            "- `matplotlib` for visualization
"
            "- access to `tests/data/hcn.xyz`

"
            "The workflow mirrors the regression test under `tests/test_tls/test_meep_2d_socket_rttddft*.py`.

## 1. Import dependencies and locate the geometry

"
            "We start by importing MaxwellLink, Meep, and plotting utilities. The RT-TDDFT"
            " driver needs the equilibrium HCN geometry; the tutorial aborts early if the"
            " file is missing.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import maxwelllink as mxl
from pathlib import Path

from maxwelllink import sockets as mxs

try:  # noqa: SIM105 - deliberate tutorial hint
    import meep as mp
except ImportError as exc:  # pragma: no cover - docs hint
    raise RuntimeError('Meep is required. Install it via conda-forge.') from exc

XYZ_PATH = Path('../tests/data/hcn.xyz').resolve()
if not XYZ_PATH.exists():
    raise FileNotFoundError(f'XYZ file not found: {XYZ_PATH}')

print(f'Loaded geometry from {XYZ_PATH}')

Using MPI version 4.1, 1 processes
Loaded geometry from /Users/taoli/Documents/Github/MaxwellLink/tests/data/hcn.xyz


## 2. Allocate the SocketHub, molecule, and Meep simulation

"
            "The SocketHub manages communication with the external RT-TDDFT driver. We couple"
            " a single MaxwellLink molecule to a 2D Meep grid and reuse the standard resolution"
            " and time-step settings employed in the test suite.

In [2]:
host, port = mxs.get_available_host_port()
hub = mxl.SocketHub(host=host, port=port, timeout=10.0, latency=1e-5)

molecule = mxl.Molecule(
    hub=hub,
    center=mp.Vector3(0, 0, 0),
    size=mp.Vector3(1, 1, 1),
    sigma=0.1,
    dimensions=2,
)

sim = mxl.MeepSimulation(
    hub=hub,
    molecules=[molecule],
    time_units_fs=0.1,
    cell_size=mp.Vector3(8, 8, 0),
    boundary_layers=[mp.PML(3.0)],
    resolution=10,
)

print(f'SocketHub listening on {host}:{port}')

[Init Molecule] Under socket mode, registered molecule with ID 0


 ######### MaxwellLink Units Helper #########
 MEEP uses its own units system, which is based on the speed of light in vacuum (c=1), 
 the permittivity of free space (epsilon_0=1), and the permeability of free space (mu_0=1). 
 To couple MEEP with molecular dynamics, we set [c] = [epsilon_0] = [mu_0] = [hbar] = 1. 
 By further defining the time unit as 1.00E-01 fs, we can fix the units system of MEEP (mu).

 Given the simulation resolution = 10,
 - FDTD dt = 5.00E-02 mu (0.5/resolution) = 5.00E-03 fs
 - FDTD dx = 1.00E-01 mu (1.0/resolution) = 3.00E+00 nm
 - Time [t]: 1 mu = 1.00E-01 fs = 4.13E+00 a.u.
 - Length [x]: 1 mu = 3.00E+01 nm
 - Angular frequency [omega = 2 pi * f]: 1 mu = 6.5851E+00 eV = 2.4200E-01 a.u.
 - Electric field [E]: 1 mu = 6.65E+07 V/m = 1.29E-04 a.u.
 Hope this helps!
 ############################################


SocketHub listening on 127.0.0.1:63835


## 3. Launch the Psi4 RT-TDDFT driver in a separate terminal

"
            "Open another shell and execute the command below. It connects the Psi4-based"
            " driver to the SocketHub created above. Once the driver reports that it is"
            " waiting for fields, return to this notebook to start the Meep simulation.

In [3]:
import shlex

driver_args = [
    'mxl_driver',
    '--model', 'rttddft',
    '--address', host,
    '--port', str(port),
    '--param',
    f'molecule_xyz={XYZ_PATH}, functional=SCF, basis=sto-3g, ' 
    'delta_kick_au=1e-1, dt_rttddft_au=0.04, electron_propagation=pc, threshold_pc=1e-6',
]
driver_cmd = shlex.join(driver_args)
print('Run the following command in a separate terminal:')
print(driver_cmd)

Run the following command in a separate terminal:
mxl_driver --model rttddft --address 127.0.0.1 --port 63835 --param 'molecule_xyz=/Users/taoli/Documents/Github/MaxwellLink/tests/data/hcn.xyz, functional=SCF, basis=sto-3g, delta_kick_au=1e-1, dt_rttddft_au=0.04, electron_propagation=pc, threshold_pc=1e-6'


## 4. Run the coupled simulation and harvest observables

"
            "With the driver running, execute the cell below to integrate the coupled"
            " Meep/RT-TDDFT system. After the run completes we extract the dipole history"
            " recorded by the driver and shut down the SocketHub.

In [ ]:
try:
    sim.run(until=80)
finally:
    hub.stop()

mu_z = np.array([entry['mu_z_au'] for entry in molecule.additional_data_history])
time_au = np.array([entry['time_au'] for entry in molecule.additional_data_history])
print(f'Captured {mu_z.size} RT-TDDFT samples.')

## 5. Compare against the reference trajectory

"
            "The repository ships a reference dipole trace for this setup. We load it,"
            " quantify the discrepancy, and visualize the simulated vs. reference signal.

In [ ]:
ref_path = Path('tests/test_psi4_rttddft/test_meep_2d_socket_rttddft_mu_z_au_ref.txt')
reference = np.loadtxt(ref_path)

if not np.allclose(time_au, reference[:, 0], atol=1e-8):
    print('Warning: time axis differs from reference data')

max_delta = np.max(np.abs(mu_z - reference[:, 1]))
print(f'Max deviation = {max_delta:.3e} a.u.')

plt.figure(figsize=(6, 4))
plt.plot(time_au, mu_z, label='Simulation')
plt.plot(reference[:, 0], reference[:, 1], '--', label='Reference')
plt.xlabel('time (a.u.)')
plt.ylabel(r'$\mu_z$ (a.u.)')
plt.legend()
plt.tight_layout()
plt.show()

### Wrap-up

"
            "You now have a complete socket-based workflow that links Meep to the Psi4"
            " RT-TDDFT driver. The same pattern extends to larger systems or different"
            " electronic-structure settings by adjusting the driver parameters printed above.